# Quick start with pinocchio

Within this script we are going to
1. create a simple URDF model by hands, and compute inertia parameters for the bodies using `trimesh`
2. Pinocchio + meshcat
3. and try command for model and data from **[Pinocchio Cheat Sheet](https://github.com/stack-of-tasks/pinocchio/blob/master/doc/pinocchio_cheat_sheet.pdf)**

# 1. URDF + trimesh

**[URDF](https://arxiv.org/pdf/2308.00514)**, Unified Robot Description Format is an XML format for representing a robot model. URDF was initially been designed ti be used in Robot Operating System (ROS) tools and Gazebo simulator, but now most of simulators uses that format.

The **[URDF](https://wiki.ros.org/urdf/XML)** model consists of links and joints motion; the default URDF can be used for open chains only, for the closed chains extra steps must be done. 

File `two_bodies.urdf` contains 3 links:
* `world` which acts as a fixed root
* `body_one` is connected with `world` via `fixed_joint`
* `body_two` is related to `world` via `floating_joint`

<b>body_one</b>
Let us pick box shape for the `body_one` and pick sizes as [.4, .2, .1] m. The sizes must be written in `geometry` container. 

In order to fill in data to the `inertial` container, we need to compute data using `trimesh`.

In [ ]:
import trimesh as tr

box_one = tr.primitives.Box(extents = [.4, .2, .1])
box_one.density = 7800 # ~steel
print(f'mass = {box_one.mass}') # [kg]
print(f'inertia = {box_one.moment_inertia}') # [kg * m**2]

Now you can fill in `mass` and `inertia` values.

<b>body_two</b>

For the second body let us choose a cylinder shape. 

In [ ]:
cylinder = tr.primitives.Cylinder(radius=.05, height=.15)
cylinder_mass = 5
cylinder.density = cylinder_mass / cylinder.volume
print(cylinder.moment_inertia)

We can also us apply coordinate transformation if it is needed. 

In [ ]:
rotation = tr.transformations.euler_matrix(1, 0, 0, axes='sxyz')
cylinder.apply_transform(rotation)
cylinder.show()
print(cylinder.moment_inertia)

Now you can fill in `mass` and `inertia` values for the second body.

# 2. Pinocchio + meshcat

In [ ]:
from os import path
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import meshcat

In [95]:
MODEL_NAME = "two_bodies.urdf"
ROOT = path.abspath('')
FOLDER_PATH = path.join(ROOT, '')
MODEL_PATH = path.join(FOLDER_PATH, MODEL_NAME)

model, collision_model, visual_model = pin.buildModelsFromUrdf(MODEL_PATH, None, pin.JointModelFreeFlyer())
data = model.createData()

In [96]:
viz = MeshcatVisualizer(model, collision_model, visual_model)
viz.viewer = meshcat.Visualizer(zmq_url="tcp://127.0.0.1:6000")
viz.clean()
# meshcat-server --open

# Load the robot in the viewer.
viz.loadViewerModel()

# Display a robot configuration.
q0 = pin.neutral(model)
viz.display(q0)
viz.displayVisuals(True)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [ ]:
hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()

# 3. Model

In [77]:
model.name

'two_bodies'

In [78]:
model.names.tolist()

['universe', 'root_joint', 'floating_joint_cylinder', 'floating_joint_sphere']

In [79]:
model.joints.tolist()

[JointModelRX
   index: 18446744073709551615
   index q: -1
   index v: -1
   nq: 1
   nv: 1,
 JointModelFreeFlyer
   index: 1
   index q: 0
   index v: 0
   nq: 7
   nv: 6,
 JointModelFreeFlyer
   index: 2
   index q: 7
   index v: 6
   nq: 7
   nv: 6,
 JointModelFreeFlyer
   index: 3
   index q: 14
   index v: 12
   nq: 7
   nv: 6]

In [80]:
model.jointPlacements.tolist()

[SE3(array([[1., 0., 0., 0.],[0., 1., 0., 0.],[0., 0., 1., 0.],[0., 0., 0., 1.]])),
 SE3(array([[1., 0., 0., 0.],[0., 1., 0., 0.],[0., 0., 1., 0.],[0., 0., 0., 1.]])),
 SE3(array([[1.  , 0.  , 0.  , 0.15],[0.  , 1.  , 0.  , 0.05],[0.  , 0.  , 1.  , 0.2 ],[0.  , 0.  , 0.  , 1.  ]])),
 SE3(array([[ 1.  ,  0.  ,  0.  , -0.15],[ 0.  ,  1.  ,  0.  , -0.05],[ 0.  ,  0.  ,  1.  ,  0.2 ],[ 0.  ,  0.  ,  0.  ,  1.  ]]))]

In [81]:
model.inertias.tolist()

[  m = 0
   c = 0 0 0
   I = 
 0 0 0
 0 0 0
 0 0 0,
   m = 62.4
   c = 0 0 0
   I = 
  0.26     0     0
     0 0.884     0
     0     0  1.04,
   m = 5
   c = 0 0 0
   I = 
 2.94524e-06           0           0
           0 2.94524e-06           0
           0           0 1.47262e-06,
   m = 5
   c = 0 0 0
   I = 
 2.94524e-06           0           0
           0 2.94524e-06           0
           0           0 1.47262e-06]

In [82]:
print(model.nq, model.nv)


21 18


# 4. Data

In [83]:
q = pin.neutral(model)
pin.framesForwardKinematics(model, data, q)

In [84]:
data.joints.tolist()

[JointDataRX, JointDataFreeFlyer, JointDataFreeFlyer, JointDataFreeFlyer]

In [85]:
data.oMi.tolist()

[SE3(array([[1., 0., 0., 0.],[0., 1., 0., 0.],[0., 0., 1., 0.],[0., 0., 0., 1.]])),
 SE3(array([[1., 0., 0., 0.],[0., 1., 0., 0.],[0., 0., 1., 0.],[0., 0., 0., 1.]])),
 SE3(array([[1.  , 0.  , 0.  , 0.15],[0.  , 1.  , 0.  , 0.05],[0.  , 0.  , 1.  , 0.2 ],[0.  , 0.  , 0.  , 1.  ]])),
 SE3(array([[ 1.  ,  0.  ,  0.  , -0.15],[ 0.  ,  1.  ,  0.  , -0.05],[ 0.  ,  0.  ,  1.  ,  0.2 ],[ 0.  ,  0.  ,  0.  ,  1.  ]]))]

In [86]:
data.oMf.tolist()

[SE3(array([[1., 0., 0., 0.],[0., 1., 0., 0.],[0., 0., 1., 0.],[0., 0., 0., 1.]])),
 SE3(array([[1., 0., 0., 0.],[0., 1., 0., 0.],[0., 0., 1., 0.],[0., 0., 0., 1.]])),
 SE3(array([[1., 0., 0., 0.],[0., 1., 0., 0.],[0., 0., 1., 0.],[0., 0., 0., 1.]])),
 SE3(array([[1., 0., 0., 0.],[0., 1., 0., 0.],[0., 0., 1., 0.],[0., 0., 0., 1.]])),
 SE3(array([[1., 0., 0., 0.],[0., 1., 0., 0.],[0., 0., 1., 0.],[0., 0., 0., 1.]])),
 SE3(array([[1.  , 0.  , 0.  , 0.15],[0.  , 1.  , 0.  , 0.05],[0.  , 0.  , 1.  , 0.2 ],[0.  , 0.  , 0.  , 1.  ]])),
 SE3(array([[1.  , 0.  , 0.  , 0.15],[0.  , 1.  , 0.  , 0.05],[0.  , 0.  , 1.  , 0.2 ],[0.  , 0.  , 0.  , 1.  ]])),
 SE3(array([[ 1.  ,  0.  ,  0.  , -0.15],[ 0.  ,  1.  ,  0.  , -0.05],[ 0.  ,  0.  ,  1.  ,  0.2 ],[ 0.  ,  0.  ,  0.  ,  1.  ]])),
 SE3(array([[ 1.  ,  0.  ,  0.  , -0.15],[ 0.  ,  1.  ,  0.  , -0.05],[ 0.  ,  0.  ,  1.  ,  0.2 ],[ 0.  ,  0.  ,  0.  ,  1.  ]]))]

In [87]:
data.v.tolist()

[  v = 0 0 0
   w = 0 0 0,
   v = 0 0 0
   w = 0 0 0,
   v = 0 0 0
   w = 0 0 0,
   v = 0 0 0
   w = 0 0 0]

In [88]:
data.a.tolist()

[  v = 0 0 0
   w = 0 0 0,
   v = 0 0 0
   w = 0 0 0,
   v = 0 0 0
   w = 0 0 0,
   v = 0 0 0
   w = 0 0 0]

In [89]:
data.f.tolist()

[  f = 0 0 0
 tau = 0 0 0,
   f = 0 0 0
 tau = 0 0 0,
   f = 0 0 0
 tau = 0 0 0,
   f = 0 0 0
 tau = 0 0 0]

In [90]:
data.M.tolist()

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,

In [91]:
data.nle.tolist()

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [92]:
data.hg

  f = 0 0 0
tau = 0 0 0

In [93]:
data.Ag

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [ ]:
data.Ig

In [97]:
LINK_NAME = "body_two"
LINK_ID = model.getBodyId(LINK_NAME)
q = pin.neutral(model)
pin.framesForwardKinematics(model, data, q)
iMd = data.oMf[LINK_ID]
print(iMd)

  R =
 0.877583  0.403423  0.259035
        0  0.540302 -0.841471
-0.479426   0.73846   0.47416
  p = 0.15 0.05  0.2



In [101]:
M = pin.SE3.Random()
print(M)


  R =
 0.303381  0.761709 -0.572502
-0.297904 -0.494881   -0.8163
-0.905104  0.418201 0.0767792
  p =  -0.809292 -0.0288221   0.153758



In [102]:
M.rotation

array([[ 0.30338094,  0.76170949, -0.5725021 ],
       [-0.29790432, -0.49488051, -0.81630037],
       [-0.90510387,  0.41820082,  0.07677923]])

In [103]:
M.translation

array([-0.80929203, -0.02882209,  0.15375837])

In [105]:
print(pin.SE3(M.rotation, M.translation))

  R =
 0.303381  0.761709 -0.572502
-0.297904 -0.494881   -0.8163
-0.905104  0.418201 0.0767792
  p =  -0.809292 -0.0288221   0.153758



In [107]:
print(M.inverse())

  R =
 0.303381 -0.297904 -0.905104
 0.761709 -0.494881  0.418201
-0.572502   -0.8163 0.0767792
  p =  0.376105   0.53788 -0.498654



In [110]:
print(M * M.inverse())

  R =
           1  5.55112e-17  3.46945e-17
 5.55112e-17            1 -5.55112e-17
 5.55112e-17 -5.55112e-17            1
  p = 1.11022e-16           0 8.32667e-17



In [115]:
print(M.action)

[[ 0.30338094  0.76170949 -0.5725021   0.07189227  0.0640386   0.12330008]
 [-0.29790432 -0.49488051 -0.81630037 -0.68584599  0.4555658  -0.02589017]
 [-0.90510387  0.41820082  0.07677923  0.24983566  0.42245691  0.64412467]
 [ 0.          0.          0.          0.30338094  0.76170949 -0.5725021 ]
 [ 0.          0.          0.         -0.29790432 -0.49488051 -0.81630037]
 [ 0.          0.          0.         -0.90510387  0.41820082  0.07677923]]


In [116]:
print(M.homogeneous)

[[ 0.30338094  0.76170949 -0.5725021  -0.80929203]
 [-0.29790432 -0.49488051 -0.81630037 -0.02882209]
 [-0.90510387  0.41820082  0.07677923  0.15375837]
 [ 0.          0.          0.          1.        ]]
